# Lesson 02: Парсинг HTML. BeautifulSoup, MongoDB

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [76]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import glob

In [77]:
main_link = 'https://hh.kz/search/vacancy?area=40&st=searchVacancy&text=web+developer&page=0'
header = {'User-Agent': 'Chrome/78.0.3904.108 Safari/537.36'}

In [78]:
def hh_parse(main_link, header):
    """This function will parse to collect the jobs data from hh.kz website"""
    jobs_data = []
    links = []
    links.append(main_link)
    response = requests.get(main_link, headers=header).text
    html = bs(response, 'lxml')

    # to check how many pages needs to be parsed from website
    try:
        pagination = html.find_all('a', attrs={'data-qa': 'pager-page'})
        count = int(pagination[-1].text)
        for i in range(count):
            link = f'https://hh.kz/search/vacancy?area=40&st=searchVacancy&text=web+developer&page={i}'
            if link not in links:
                links.append(link)
    except:
        pass

    # to parse each page to collect the data
    for link in links:
        response = requests.get(link, headers=header).text
        html = bs(response, 'lxml')
        divs = html.find_all('div', attrs={'data-qa': 'vacancy-serp__vacancy'})
        for div in divs:
            try:
                # to parse name of vacancy
                vacancy_name = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}).text
                # to parse how much salary is
                vacancy_salary = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'})
                # here will be divided to min and max level salary as well the currency
                if vacancy_salary == None:
                    min_salary= 'None'
                    max_salary= 'None'
                    salary_currency = 'None'
                else:
                    vacancy_salary = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}).text.split(' ')
                    salary_currency = vacancy_salary[-1]
                    if vacancy_salary[0] == 'от':
                        min_salary = ''.join(vacancy_salary[1:-1])
                        max_salary = None
                    elif vacancy_salary[0] == 'до':
                        min_salary = ''.join(vacancy_salary[1:-1])
                        max_salary = None
                    else:
                        min_salary = vacancy_salary[0].split('-')[0]
                        max_salary = vacancy_salary[0].split('-')[1]

                # to parse link of the vacancy
                vacancy_link = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})['href']
                # to parse name of company
                company_name = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-employer'}).text

                # to collect all in 1 list
                jobs_data.append({
                    'vacancy_name': vacancy_name,
                    'company_name': company_name,
                    'min_salary': min_salary,
                    'max_salary': max_salary,
                    'salary_currency': salary_currency,
                    'vacancy_link': vacancy_link,
                })
                hh_data = pd.DataFrame(jobs_data) # to convert from list to DataFrame
                hh_data.to_csv('hh_data.csv', encoding='utf-8', index=False)
            except:
                pass

In [79]:
hh_parse(main_link, header)

In [80]:
main_link = 'https://www.superjob.ru/vacancy/search/?keywords=web%20developer&geo%5Bt%5D%5B0%5D=4&page=1'

In [81]:
def sj_parse(main_link):
    """This function will parse to collect the jobs data from superjob.ru website"""
    jobs_data = []
    links = []
    links.append(main_link)
    response = requests.get(main_link).text
    html = bs(response, 'lxml')

    try: # to check how many pages needs to be parsed from website
        pagination = html.find_all('span', attrs={'class': 'qTHqo _2h9me DYJ1Y _2FQ5q _2GT-y'})
        count = int(pagination[-2].text)
        for i in range(count):
            link = f'https://www.superjob.ru/vacancy/search/?keywords=web%20developer&geo%5Bt%5D%5B0%5D=4&page={i+1}'
            if link not in links:
                links.append(link)
    except:
        pass

    # to parse each page to collect the data
    for link in links:
        response = requests.get(link).text
        html = bs(response, 'lxml')
        divs = html.find_all('div', attrs={'class': '_3syPg _3P0J7 _9_FPy'})

        for div in divs:
            try:
                # to parse name of vacancy
                vacancy_name = div.find('div', attrs={'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).text
                company_name = div.find('span', {'class': '_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'})
                if company_name == None:
                    company_name = 'None'
                else:
                    company_name = div.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'}).text
                # to parse how much salary is and after will be divided to min and max level salary as well the currency
                try:
                    vacancy_salary = div.find('span', attrs={'class': '_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).text
                except:
                    vacancy_salary = None
                if vacancy_salary != None:
                    if vacancy_salary == 'По договорённости':
                        min_salary = None
                        max_salary = None
                        salary_currency = None
                    elif '—' in vacancy_salary:
                        vacancy_salary = vacancy_salary.split('—')
                        salary_currency = vacancy_salary[1][-1]
                        max_salary = vacancy_salary[1][:-1]
                        min_salary = vacancy_salary[0]
                    elif 'от' in vacancy_salary:
                        vacancy_salary = vacancy_salary.split()
                        salary_currency = vacancy_salary[-1]
                        max_salary = None
                        min_salary = ' '.join(vacancy_salary[1:-1])
                    elif 'до' in vacancy_salary:
                        vacancy_salary = vacancy_salary.split()
                        salary_currency = vacancy_salary[-1]
                        max_salary = ' '.join(vacancy_salary[1:-1])
                        min_salary = None
                    else:
                        vacancy_salary = vacancy_salary.split()
                        salary_currency = vacancy_salary[-1]
                        max_salary = ' '.join(vacancy_salary[:-1])
                        min_salary = ' '.join(vacancy_salary[:-1])
                else:
                    min_salary, max_salary, salary_currency = [None] * 3

                # to parse link of the vacancy
                vacancy_link = 'https://www.superjob.ru' + div.find('div', attrs={'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).findParent('a')['href']

                # to collect all in 1 list
                jobs_data.append({
                    'vacancy_name': vacancy_name,
                    'company_name': company_name,
                    'min_salary': min_salary,
                    'max_salary': max_salary,
                    'salary_currency': salary_currency,
                    'vacancy_link': vacancy_link,
                })

                sj_data = pd.DataFrame(jobs_data)  # to convert from list to DataFrame
                sj_data.to_csv('sj_data.csv', encoding='utf-8', index=False)
            except:
                pass

In [82]:
sj_parse(main_link)

In [83]:
df = pd.concat(map(pd.read_csv, glob.glob('*data.csv')))

In [84]:
df

,vacancy_name,company_name,min_salary,max_salary,salary_currency,vacancy_link
0,Web-разработчик (программист),ТОО IMARKETING,200 000,600 000,KZT,https://hh.kz/vacancy/35514596?query=web%20dev...
1,Frontend-разработчик,ТОО CapStone,100 000,NaN,KZT,https://hh.kz/vacancy/35174088?query=web%20dev...
2,ВEБ-РАЗРАБОТЧИК,ТОО B1.Business,200 000,NaN,KZT,https://hh.kz/vacancy/35205048?query=web%20dev...
3,Backend Python Developer,ТОО AVIATA.KZ,750 000,NaN,KZT,https://hh.kz/vacancy/35395690?query=web%20dev...
4,Аsp.net developer,ТОО Norsec Delta Projects,None,None,None,https://hh.kz/vacancy/35179129?query=web%20dev...
...,...,...,...,...,...,...
11,Программист Linux / Web-разработчик,МЕДИКУМ,NaN,NaN,NaN,https://www.superjob.ru/vakansii/programmist-l...
12,Программист (web-разработчик),Государственное учреждение сектора госуправления,NaN,NaN,NaN,https://www.superjob.ru/vakansii/programmist-3...
13,ABAP-разработчик,"Глобус, Сеть гипермаркетов",NaN,NaN,NaN,https://www.superjob.ru/vakansii/abap-razrabot...
14,"Full-Stack developer (Angular, C#, .NET)",«Группа ГУТА»,NaN,190 000,₽,https://www.superjob.ru/vakansii/full-stack-de...
